In [1]:
import SimpleITK as sitk
from DataHandler import DataHandler
import pickle
import numpy as np
import os

In [2]:
moving_image = sitk.ReadImage("/home/cschellenberger/datam2olie/synthetic/native/t1/Synthetic_CT/CT_Model71_Energy100_atn_1.nrrd")

In [4]:
max(sitk.GetArrayFromImage(moving_image))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [2]:
def get_moved_points(points: np.array, displacement: sitk.Image) -> np.array:
    displacement_copy = displacement.__copy__()
    displacement_transform = sitk.DisplacementFieldTransform(displacement_copy)
    moved_points = [displacement_transform.TransformPoint(point) for point in points]
    return moved_points

def get_landmarks(fixed_image_path: str, indexing: str = 'zyx', continuous = False, preResample = False):
    model_name = os.path.basename(fixed_image_path).replace('_atn_3.nrrd', '')
    if preResample: 
        loaded_points = pickle.load(open(f'/home/cschellenberger/Documents/vectorPickles/{model_name}_vec_frame1_to_frame2.p', "rb"))
        moving_landmarks = np.array([(float(loaded_points[idx]['1X']), float(loaded_points[idx]['1Y']), float(loaded_points[idx]['1Z'])) for (idx, _) in enumerate(loaded_points)])
        fixed_landmarks = np.array([(float(loaded_points[idx]['2X']), float(loaded_points[idx]['2Y']), float(loaded_points[idx]['2Z'])) for (idx, _) in enumerate(loaded_points)])
    else: 
        if continuous: loaded_points = pickle.load(open(f'/home/cschellenberger/Documents/vectorPickles/CT_points_t1_t3_withRegion_Spacing1_8/{model_name}_idx.p', "rb"))
        else: loaded_points = pickle.load(open(f'/home/cschellenberger/Documents/vectorPickles/CT_points_t1_t3_withRegion/{model_name}_idx.p', "rb"))
        moving_landmarks = np.array(loaded_points['t1'])
        fixed_landmarks = np.array(loaded_points['t3'])
    regions = np.array(loaded_points['Region'])
    if indexing == 'xzy':
        # swap columns because numpy and vxm use zyx indexing and the data uses xyz indexing
        moving_landmarks[:, [1, 2]] = moving_landmarks[:, [2, 1]]
        fixed_landmarks[:, [1, 2]] = fixed_landmarks[:, [2, 1]]
    elif indexing == 'zyx':
        # swap columns because numpy and vxm use zyx indexing and the data uses xyz indexing
        moving_landmarks[:, [0, 2]] = moving_landmarks[:, [2, 0]]
        fixed_landmarks[:, [0, 2]] = fixed_landmarks[:, [2, 0]]
    elif indexing == 'yxz':
        # swap columns because numpy and vxm use zyx indexing and the data uses xyz indexing
        moving_landmarks[:, [0, 1]] = moving_landmarks[:, [1, 0]]
        fixed_landmarks[:, [0, 1]] = fixed_landmarks[:, [1, 0]]
    else: assert indexing == 'xyz', f'indexing can only be xyz or zyx. Got: {indexing}'
    return moving_landmarks.astype(np.float64), fixed_landmarks.astype(np.float64), regions

In [3]:
idx = 0
dh = DataHandler(val_images=12)
# dh.get_synthetic_data(
#     fixed_path='/home/cschellenberger/datam2olie/synthetic/orig/t3/Synthetic_CT/',
#     moving_path='/home/cschellenberger/datam2olie/synthetic/orig/t1/Synthetic_MR/')
dh.get_synthetic_data(
    fixed_path='/home/cschellenberger/Documents/newT3Resample1_8',
    moving_path='/home/cschellenberger/Documents/T1ResampledSpacing1_8')
moving_image_paths = dh.x_val
fixed_image_paths = dh.y_val
resErr = 0
dists = {}
c = 0
for idx in range(1): #len(moving_image_paths)
    moving_image = sitk.ReadImage(moving_image_paths[idx])
    fixed_image = sitk.ReadImage(fixed_image_paths[idx])
    elastix_image_filter = sitk.ElastixImageFilter()
    elastix_image_filter.SetFixedImage(fixed_image)
    elastix_image_filter.SetMovingImage(moving_image)
    parameterMap0 = sitk.ReadParameterFile('./params.txt')
    elastix_image_filter.SetParameterMap(parameterMap0)
    moving_landmarks, fixed_landmarks, regions = get_landmarks(fixed_image_paths[idx], indexing='xyz', continuous = True)
    fixed_landmarks_tmp, moving_landmarks_tmp, regions_tmp = [], [], []
    for j in range(len(fixed_landmarks)):
        if fixed_landmarks[j][2] > 12 and fixed_landmarks[j][2] < 116:
            fixed_landmarks_tmp.append(fixed_landmarks[j])
            moving_landmarks_tmp.append(moving_landmarks[j])
            regions_tmp.append(regions[j])
    fixed_landmarks = np.array(fixed_landmarks_tmp)
    moving_landmarks = np.array(moving_landmarks_tmp)
    regions = regions_tmp
    # liver = [1 if regions[i] == 'known_vector_liver' else 0 for i in range(len(fixed_landmarks))] # or regions[i] == 'known_vector_stomach' 
    # liver_fixed = sitk.Image(fixed_image.GetSize(), sitk.sitkUInt8)
    # liver_fixed.SetOrigin(fixed_image.GetOrigin())
    # liver_fixed.SetSpacing(fixed_image.GetSpacing())
    # for i, point in enumerate(fixed_landmarks):
    #     try: liver_fixed.SetPixel(int(point[0]), int(point[1]), int(point[2]), int(liver[i]))
    #     except: continue
    # bdi = sitk.BinaryDilateImageFilter()
    # bdi.SetForegroundValue(1)
    # #bdi.SetKernelRadius((4,4,16))
    # liver_fixed_bdi = bdi.Execute(liver_fixed)
    # elastix_image_filter.SetFixedMask(liver_fixed_bdi)
    elastix_image_filter.Execute()
    transformixFilter = sitk.TransformixImageFilter()
    transformixFilter.SetTransformParameterMap(elastix_image_filter.GetTransformParameterMap())
    transformixFilter.ComputeDeformationFieldOn()
    transformixFilter.SetMovingImage(moving_image)
    transformixFilter.Execute()
    moved = transformixFilter.GetResultImage()
    displacement = sitk.GetImageFromArray(sitk.GetArrayFromImage(transformixFilter.GetDeformationField()) / 1.8) 
    displacement = sitk.Cast(displacement, sitk.sitkVectorFloat64)
    fixed_landmarks_tmp, moving_landmarks_tmp = [], []
    moved_landmarks = get_moved_points(fixed_landmarks, displacement)
    err = moving_landmarks - moved_landmarks
    err = np.linalg.norm(err, axis = 1)
    resErr += err.mean()
    dists[f'dist{idx}'] = err.mean()
print(resErr / len(moving_image_paths))
#pickle.dump(dists, open("./elastixResultsS2.p", "wb"))


Installing all components.
InstallingComponents was successful.

  The default value "FixedSmoothingImagePyramid" is used instead.
  The default value "MovingSmoothingImagePyramid" is used instead.
ELASTIX version: 5.000
Command line options from ElastixBase:
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ./
-threads  unspecified, so all available threads are used
Command line options from TransformBase:
-t0       unspecified, so no initial transform used
  The default value "3" is used instead.
  The default value "false" is used instead.

Reading images...
Reading images took 0 ms.

  A default pyramid schedule is used.
  A default pyramid schedule is used.
Initialization of all components (before registration) took: 2 ms.
Preparation of the image pyramids took: 282 ms.

Resolution: 0
  The default value "false" is used instead.
  The default value "true" is used instead.
  The default value "true" is used instead.
Setting the fix

In [14]:
moving_landmarks_liver = np.array([moving_landmarks[i] for i in range(len(moving_landmarks)) if regions[i] == 'known_vector_liver'])
moved_landmarks_liver = np.array([moved_landmarks[i] for i in range(len(moved_landmarks)) if regions[i] == 'known_vector_liver'])
err_liv = moving_landmarks_liver - moved_landmarks_liver
err_liv = np.linalg.norm(err_liv, axis = 1)
err_liv.mean()

14.952151450033716

In [13]:
err.mean()

9.044722305785472

In [2]:
dists = pickle.load(open("./elastixResultsnc.p", "rb"))
print(dists)
print(np.array(list(dists.values())).mean())

{'dist0': 4.888210385645453, 'dist1': 4.273592699811264, 'dist2': 5.44875690019965, 'dist3': 5.5805299343092445, 'dist4': 5.4649818885880235, 'dist5': 5.124764793868303, 'dist6': 5.522443092375467, 'dist7': 4.330270327066274, 'dist8': 6.7657790722544355, 'dist9': 5.890818623782708, 'dist10': 4.774737987087083, 'dist11': 6.873447459989878}
5.41152776374815


In [2]:
dists = pickle.load(open("./elastixResults.p", "rb"))
print(dists)
print(np.array(list(dists.values())).mean())

{'dist0': 4.856176112053943, 'dist1': 4.236578683363468, 'dist2': 5.419552849818011, 'dist3': 5.5548284600450275, 'dist4': 5.4420299539727175, 'dist5': 5.095367754454074, 'dist6': 5.50081818075844, 'dist7': 4.300471473360043, 'dist8': 6.739431798839116, 'dist9': 5.8665376784753445, 'dist10': 4.749709234287543, 'dist11': 6.850792937438295}
5.384357926405502


In [18]:
dists = pickle.load(open("./elastixResults1_8.p", "rb"))
print(dists)
print(np.array(list(dists.values())).mean() * 1.8)

{'dist0': 3.8035425002983203, 'dist1': 4.1284505710966055, 'dist2': 4.6737168509997815, 'dist3': 5.294959051460766, 'dist4': 4.565779951468382, 'dist5': 4.6945081225186325, 'dist6': 4.854044332244038, 'dist7': 4.29674435061808, 'dist8': 5.669503744236139, 'dist9': 5.484753404214473, 'dist10': 4.238744601681116, 'dist11': 5.287562256587489}
8.548846460613575


In [17]:
dists = pickle.load(open("./elastixResultsLiverMask.p", "rb"))
print(dists)
print(np.array(list(dists.values())).mean() * 2)

{'dist0': 3.3263505344723403, 'dist1': 3.9459189373589836, 'dist2': 4.3403315396069635, 'dist3': 4.957021951182022, 'dist4': 4.232735893636641, 'dist5': 4.571790309725407, 'dist6': 4.593514793408607, 'dist7': 3.99903083959989, 'dist8': 4.91698374276908, 'dist9': 5.160597428474063, 'dist10': 4.040792247755303, 'dist11': 5.840434930917136}
8.987583858151073


In [ ]:
liver_fixed = sitk.AntiAliasBinaryImageFilter().Execute(liver_fixed) 

In [35]:
moving_landmarks, fixed_landmarks, regions = get_landmarks(fixed_image_paths[0], indexing='xyz', continuous = True)
liver = [1 if regions[idx] == 'known_vector_liver' or regions[idx] == 'known_vector_stomach'  else 0 for idx in range(len(fixed_landmarks))]
liver_fixed = sitk.Image(fixed_image.GetSize(), sitk.sitkUInt8)
liver_fixed.SetOrigin(fixed_image.GetOrigin())
liver_fixed.SetSpacing(fixed_image.GetSpacing())
for idx, point in enumerate(fixed_landmarks):
    try: liver_fixed.SetPixel(int(point[0]), int(point[1]), int(point[2]), int(liver[idx]))
    except: continue
bdi = sitk.BinaryDilateImageFilter()
bdi.SetForegroundValue(1)
liver_fixed_bdi = bdi.Execute(liver_fixed)

In [11]:
from pyM2aia import M2aiaOnlineHelper
moving_image = sitk.ReadImage(moving_image_paths[0])
fixed_image = sitk.ReadImage(fixed_image_paths[0])
M2aiaHelper = M2aiaOnlineHelper("ipynbViewer", "jtfc.de:5050/m2aia/m2aia-no-vnc:with_exit", "8899")
with M2aiaHelper as helper:
    helper.show({"moved": moved, "liver_bdi": liver_fixed_bdi, "Moving": moving_image, "Fixed": fixed_image,}) # "Moved": moved

You can find your images @  http://141.19.142.80:8899

